# Exploring Python's Built-in Exception Hierarchy

## Why This Matters

Python ships with a **rich hierarchy of exception classes**. Knowing a relevant subset helps you write clearer, safer code and debug faster.

### Key ideas

| Idea | Explanation |
|------|-------------|
| **Single base** | All ordinary runtime exceptions are **subclasses of `Exception`**. |
| **Meaning over noise** | Subclasses convey **why** something failed, not just that "something went wrong." |
| **Avoid masking bugs** | Catching broad bases like `Exception` hides root causes and can mask real bugs. |
| **Prefer narrow `except`** | Narrow the exception class so you handle only what you intend and can react appropriately. |
| **Inheritance = flexibility** | The tree tells you when one `except` can cover many related errors vs. when to be specific. |

### What we'll do

We'll **inspect the exception tree** that comes with Python using a small recursive helper, then look at one level and three levels deep to see the structure.

## Imports and the `show_tree` Function

We use two modules:

- **`inspect`** — for introspecting objects (e.g. checking if something is a class).
- **`builtins`** — the module that holds all built-in names; exception classes live here.

### How `show_tree` works

The function **recursively** prints the exception class hierarchy under a given base class.

| Parameter | Purpose |
|-----------|---------|
| `base` | The exception class to use as the root (e.g. `Exception`). |
| `level` | Current depth (default `0`); used for indentation and to stop recursion. |
| `max_depth` | Stop recursing when `level > max_depth`. |

**Algorithm (high level):**

1. **Base case:** If `level > max_depth`, return (stop recursion).
2. **Scan builtins:** Loop over `vars(builtins).items()` to get every name and object in the built-in namespace.
3. **Filter:** Keep only objects that are **classes**, are **subclasses of `base`**, and are **not `base` itself** (avoids infinite recursion).
4. **Print:** Print the class name with indentation (`"\t" * level`).
5. **Recurse:** Call `show_tree(obj, level + 1, max_depth)` so each subclass becomes the new root for the next level.

This gives a simple text "tree" of exception types directly available in Python.

In [ ]:
import inspect
import builtins

def show_tree(base, level=0, max_depth=1):
    """Recursively print the exception class hierarchy under `base` up to `max_depth`."""
    if level > max_depth:
        return
    print("--------------------------------")
    print("The line below is the builtins module")
    print(vars(builtins))
    print("--------------------------------")
    print("The line below is the builtins module items")
    print(vars(builtins).items())
    print("--------------------------------")
    for name, obj in vars(builtins).items():
        if inspect.isclass(obj) and issubclass(obj, base) and obj is not base:
            print("\t" * level + f"{name}")
            show_tree(obj, level + 1, max_depth)

### What does `vars()` do?

We use `vars(builtins)` to get everything that lives in the built-in namespace.

- **With an argument:** `vars(object)` returns the **`__dict__`** of that object — the dictionary that holds its attributes.
- **For a module:** So `vars(builtins)` is a dictionary of **all names** (and their values) in the `builtins` module — functions, types, exception classes, etc.

That’s why we iterate with `vars(builtins).items()`: we need to consider every built-in name and then filter down to exception classes. Run the cell below to see the docstring with `vars?`.

In [2]:
# Optional: see the docstring for vars
vars?

Docstring:
Show vars.

Without arguments, equivalent to locals().
With an argument, equivalent to object.__dict__.
Type:      builtin_function_or_method

### What does `inspect.isclass()` do?

Not everything in `builtins` is a class (e.g. `print`, `len`, `True`). We only want **exception classes**.

- **`inspect.isclass(obj)`** returns `True` if `obj` is a class (including built-in types and exception types).
- Using it in our loop **filters out** functions, constants, and other attributes so we only recurse into class hierarchies.

Together with `issubclass(obj, base)` and `obj is not base`, we restrict the tree to real exception subclasses of our base and avoid infinite recursion. Run the cell below for the full docstring.

In [3]:
# Optional: see the docstring for inspect.isclass
help(inspect.isclass)

Help on function isclass in module inspect:

isclass(object)
    Return true if the object is a class.



## Exception Tree with `max_depth=1`

Here we show **only the direct subclasses** of `Exception` (one level deep).

**What you’ll see (examples):**

- **`ArithmeticError`** — numeric errors (division by zero, overflow, etc.).
- **`AssertionError`** — failed `assert` statements.
- **`AttributeError`** — attribute get/set on an object that doesn’t support it.
- **`OSError`** — I/O and OS-related errors (files, network, etc.); very common.
- **`RuntimeError`** — generic “something went wrong at runtime.”
- **`SyntaxError`** — invalid Python syntax.
- **`ValueError`** — wrong value type/range (e.g. `int("abc")`).
- …and others.

This is the **top level** of the built-in exception hierarchy. Run the cell below to print the full list.

In [4]:
show_tree(Exception, max_depth=1)

{'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in f

## Exception Tree with `max_depth=3`

With **`max_depth=3`** we go **three levels** deep. Indentation shows parent → child.

**Examples of what appears:**

- **Under `ArithmeticError`:** e.g. `FloatingPointError`, `ZeroDivisionError` — more specific numeric errors.
- **Under `OSError`:** e.g. `FileNotFoundError`, `PermissionError`, `ConnectionError` — specific I/O and OS failures.
- **Under `LookupError`:** e.g. `IndexError`, `KeyError` — indexing and key access.

This view shows how **exception families** are organized: you can catch `OSError` to handle all OS-related errors, or catch `FileNotFoundError` when you only care about missing files. Run the cell below to see the full tree.

In [ ]:
show_tree(Exception, max_depth=3)

---

## Summary

- We used **`inspect`** and **`builtins`** with a recursive **`show_tree()`** to print the built-in exception hierarchy.
- **`max_depth=1`** shows direct children of `Exception`; **`max_depth=3`** shows deeper families (e.g. `OSError` → `FileNotFoundError`).
- This is only what’s **built into Python**; libraries add their own exceptions, but the same idea applies: use the hierarchy to catch at the right level.

**Next steps:** Use this picture to catch **specific** exceptions (e.g. `FileNotFoundError`, `ValueError`) instead of bare `Exception`, and to explore exception families and handle them more explicitly in your code.